<a href="https://colab.research.google.com/github/stogaja/Tanzanian-Water-Project/blob/main/TANZANIA_WATER_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. **Defining the Question** 

Tanzania is the largest country in East Africa, with a population of 52 million people. But of those 52 million people, 23 million have no choice but to drink dirty water from unsafe sources. 44 million do not have access to adequate sanitation and 4000 children die from preventable diseases due to unsafe water. Safe water is scarce, and often women and children have to spend two to seven hours collecting clean water (WaterAid, 2016). This is quite the predicament. Water is a basic need and right for all human beings. The Tanzanian Ministry of water agrees and together with Taarifa, they aim to improve sanitation conditions in their country.
Water is fundamental to life and the environment; it plays a central role in both, economic and social development activities. Water touches all the spheres of human life including domestic, livestock, fisheries, wildlife, industry and energy, recreation, and other social—economic activities. It plays a pivotal role in poverty alleviation through the enhancement of food security, domestic hygiene, and the environment. The availability of safe and clean water raises the standard of living while its inadequacy of it poses serious health risks and leads to a decline in the living standards and life expectancy. Major fresh water sources in Tanzania include lakes, rivers, streams, dams, and groundwater. However, these are not well distributed all over the country. Some areas lack both surface and groundwater sources. Increasing population growth and urbanization pose serious pressure on the quantity and quality of available water. The sustainability of the present and future human life and environment depends mainly on proper water resources management. 


### a) Specifying the Question

Water supply to different parts of Tanzania is mainly done through pipes dug underground, while this is an initiative to curb the water problem, over 24 million people are still impacted by the crisis, that’s almost half of the population. This has resulted in poor sanitation, lack of safe drinking water as well as overcrowding at water sources, the adverse effects include disease outbreaks and generally very slow economic growth. The project aims to solve these problems by predicting which pipes are operating well, which ones need repairs and which ones are not working at all, as optimally functioning pipes will mean smooth delivery of water to where its needed.

### b) Defining the Metric for Success

The project will be considered a success when we can classify pumps into 3 categories namely:

* functional : the waterpoint is operational and there are no repairs needed

* functional needs repair : the waterpoint is operational, but needs repairs

* non functional : the waterpoint is not operational

### c) Understanding the context

### d) Recording the Experimental Design

## e) Data Relevance

The data has been proven to be valid and was provided by the Tanzania Water Ministry

# **2. Importing Libraries.**

In [ ]:
!pip install category_encoders

In [ ]:
# Importing the necessary libraries
#
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import category_encoders as ce
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
import numpy as np
import os

#  **3. Reading the Data**

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Accessing working directory
os.chdir('/content/drive/Shared drives/Final Project Group 2')

In [ ]:
#Loading the csv files
train_labels = pd.read_csv("Training set labels.csv")
train_values = pd.read_csv("Training set values.csv")
test_labels = pd.read_csv("SubmissionFormat.csv")
test_values = pd.read_csv("Test set values.csv")

In [ ]:
# Merging the Training dataset
training_data = train_labels.merge(train_values, left_on='id', right_on='id')
training_data.head()

In [ ]:
training_data.shape

In [ ]:
# Merging the Testing dataset
testing_data = test_labels.merge(test_values, left_on='id', right_on='id')
testing_data.head()

In [ ]:
testing_data.shape

In [ ]:
# Combining training and testing dataset
# For purposes of cleaning and EDA
dataset = pd.concat([training_data, testing_data], ignore_index=True, sort=False)
dataset.head()

In [ ]:
dataset.shape

In [ ]:
# Saving the dataset
dataset.to_csv("merged_dataset.csv")

In [ ]:
# Reading the dataset
df = pd.read_csv("merged_dataset.csv")

# Exploring the data

In [ ]:
#checking for shape 
# size of the dataset
print("The dataset consist of",df.shape[0], "rows and", df.shape[1], "columns")

In [ ]:
#a preview of the data 
df.head()

In [ ]:
#checking for colum names
df.columns

* amount_tsh : Total static head (amount water available to waterpoint)

* date_recorded : The date the row was entered

* funder : Who funded the well

* gps_height : Altitude of the well

* installer : Organization that installed the well
* longitude : GPS coordinate

* latitude : GPS coordinate

* wpt_name : Name of the waterpoint if there is one

* num_private :Private use or not

* basin : Geographic water basin

* subvillage : Geographic location

* region : Geographic location

* region_code : Geographic location (coded)

* district_code : Geographic location (coded)

* lga : Geographic location

* ward : Geographic location

* population : Population around the well

* public_meeting : True/False

* recorded_by : Group entering this row of data

* scheme_management : Who operates the waterpoint

* scheme_name : Who operates the waterpoint

* permit : If the waterpoint is permitted

* construction_year : Year the waterpoint was constructed

* extraction_type : The kind of extraction the waterpoint uses

* extraction_type_group : The kind of extraction the waterpoint uses

* extraction_type_class : The kind of extraction the waterpoint uses

* management : How the waterpoint is managed

* management_group : How the waterpoint is managed

* payment : What the water costs

* payment_type : What the water costs

* water_quality : The quality of the water

* quality_group : The quality of the water

* quantity : The quantity of water
quantity_group : The quantity of water

* source : The source of the water

* source_type : The source of the water

* source_class : The source of the water

* waterpoint_type : The kind of waterpoint

* waterpoint_type_group : The kind of waterpoint

In [ ]:
#cheking for data types if each columns 
df.dtypes

#  **4. Data Preparation**

# Data Cleaning.

### a)Validity

In [ ]:
# Preview sample of 100 records to see whether all records are appropiately ordered
df.sample(10)

In [ ]:
# dropping unnecesary columns
df = df.drop(columns="Unnamed: 0")

### c) Uniformity

In [ ]:
#checking if columns are properly named 
df.columns

Columns have uniform naming.

### d) Completeness

In [ ]:
# here we check for missing values 
# Dealing with missing values 
# Checking the mumber of missing values by column and sorting for the smallest

Total = df.isnull().sum().sort_values(ascending=False)

# Calculating percentages
percent_1 = df.isnull().sum()/df.isnull().count()*100

# rounding off to one decimal point
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)

# creating a dataframe to show the values
missing_data = pd.concat([Total, percent_2], axis=1, keys=['Total', '%'])
missing_data

In [ ]:
# let's replace the 0 from the construction year  with and arbitrarily selected year 1993

df['construction_year'] = df['construction_year'].replace({0:1993})
df['age'] = df['date_recorded'].astype(str).str[:4].astype(int) - df['construction_year']
df['pop/year'] = df['population'].replace({0:1}) / df['age'].replace({0:1})

The cleaned construction year enables us to engineer a pump and the average population served per year feature.

In [ ]:
# #The cunstruction year should be a datetime data type
# df['construction_year']=df['construction_year'].astype('datetime64[ns]')
# df.dtypes

In [ ]:
# we impute the missing values with the string "No Record"
df.scheme_name= df.scheme_name.fillna('No Record')
df.scheme_management = df.scheme_management.fillna('No Record')
df.installer = df.installer.fillna('No Record')
df.funder = df.funder.fillna('No Record')
df.public_meeting = df.public_meeting.fillna('No Record')
df.permit = df.permit.fillna('No Record')
df.subvillage = df.subvillage.fillna('No Record')

In [ ]:
# Checking for missing values
print(df.isnull().sum())

### e) Consistency

In [ ]:
# Check for duplicates
df.duplicated().sum()

No duplicate rows were found in our data set

**Exploratory Data Analysis**

a)Univariate analysis.

In [ ]:
df.describe()

In [ ]:
# selecting object datatypes columns
categorical = ['basin', 'region', 
         'public_meeting', 'recorded_by',
       'scheme_management', 'permit',
       'extraction_type_group', 'extraction_type_class',
       'management', 'management_group',  'payment_type',
        'quality_group', 'quantity_group',
       'source', 'source_type', 'source_class', 
       'waterpoint_type_group']
categorical

# lets make a for loop to make countplots for our categorical variables.
for col in categorical:
  ax=sns.countplot(y=col,data=df)
  plt.title(f"countplot of {col}")
  plt.show()


b)Bivariate analysis

In [ ]:
#creating a crosstab 
crosstb=pd.crosstab(df.region,df.extraction_type_class)

#creating a bar plot
plt.figure(figsize=(34,30))
pl=crosstb.plot(kind="bar",stacked=True,rot=90)
plt.title("extraction mode in each region")
plt.show()

Plot above shows region and most used mode of extraction on the water pumps



In [ ]:
#creating a crosstab 
crosstb=pd.crosstab(df.region,df.payment_type)

#creating a bar plot
plt.figure(figsize=(34,30))
pl=crosstb.plot(kind="bar",stacked=True,rot=90)
plt.title("payment criteria per region")
plt.show()

Plot above shows how pple pay for their water ,we can see that in Dar es salaam and mtwara payment per backet is almost more common than never pay ,meaning its harder to get free water than all other places.

In [ ]:
#creating a crosstab 
crosstb=pd.crosstab(df.region,df.source_type)

#creating a bar plot
plt.figure(figsize=(34,30))
pl=crosstb.plot(kind="bar",stacked=True,rot=90)
plt.title("source type in each region")
plt.show()

Springs seem to be a commomn source across the regions followed by shallow well and river/lake and boreholes.
The city Dar es salam seems to have very few water sources.

In [ ]:
#creating a crosstab 
crosstb=pd.crosstab(df.region,df.management_group)

#creating a bar plot
plt.figure(figsize=(34,30))
pl=crosstb.plot(kind="bar",stacked=True,rot=90)
plt.title("management group in each region")
plt.show()

Most pumps seem to be managed by the comunities that use them .

Performing Feature Engineering

In [ ]:
# for df
df['water_/_person'] = df['amount_tsh'].replace({0:1}) / df['population'].replace({0:1})

We will then write a function to check for the cardinality of each feature(how many unique values there are in the feature)

In [ ]:
def reverse_cardinality_check(n, df):
# this function will search the dataframe for features above the cardinality limit, 
# then create a dict from the results
  
  feature_list = []
  
  cardinality_value = []
  
  for _ in range(len(df.columns)):
    if len(df[df.columns[_]].value_counts()) > n:
      
      feature_list.append(df.columns[_])
      
      cardinality_value.append(len(df[df.columns[_]].value_counts()))
                               
        
  feature_dict = dict(zip(feature_list, cardinality_value))
  
  return feature_dict

We will then preview our high cardinality features

In [ ]:
high_cardinality_feature_dict = reverse_cardinality_check(150, df)
high_cardinality_feature_dict

We will create dataframes for our high and low cardinality features

In [ ]:
# dataframe for high cardinality
high_cardinality_features = df[list(high_cardinality_feature_dict.keys())]
high_cardinality_features.columns

In [ ]:
# dataframe for low cardinality features
low_cardinality_features = df.drop(columns = list(high_cardinality_feature_dict.keys()))
low_cardinality_features.columns

Let us now perform label encoding for each dataframe

In [ ]:
# Encoding the numerical columns
one_hot_encode = ce.OneHotEncoder(use_cat_names=True)
one_hot_encode.fit(low_cardinality_features, df['status_group'])
low_cardinality_features = one_hot_encode.transform(low_cardinality_features)

ordinal_encode = ce.OrdinalEncoder()
ordinal_encode.fit(high_cardinality_features, df['status_group'])
high_cardinality_features = ordinal_encode.transform(high_cardinality_features)

In [ ]:
high_cardinality_features.isnull().sum()

In [ ]:
# let's drop missing values in the encoded dataframes
low_cardinality_features.dropna().head()
high_cardinality_features.dropna().head()

We have null values in the index columns

Let's merge the two encoded features back together

In [ ]:
# features = low_cardinality_features.concat(high_cardinality_features,
#                                           on = low_cardinality_features.index)
frames =[low_cardinality_features, high_cardinality_features]

features = pd.concat(frames, axis = 1)

In [ ]:
# previewing the datatset
features.head()

Next we impute and scale our features

In [ ]:
# let's impute using the mean
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(features, df['status_group'])
features = imputer.transform(features)

scaler = RobustScaler()
scaler.fit(features, df['status_group'])
features = scaler.transform(features)

In [ ]:
df.columns

In [ ]:
# let's select our x and y variables
X = df.drop('status_group', axis = 1).values
y = df['status_group']

from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

Let's perform the operations above for our target dataframe as well(x Test)

In [ ]:
# # for high cardinality columns in the test dataset
# high_cardinality_X_test_dict = reverse_cardinality_check(150, X_test)
# high_cardinality_X_test_dict

In [ ]:
# low_cardinality_X_test = X_test.drop(columns = list(high_cardinality_X_test_dict.keys()))
# low_cardinality_X_test.columns

In [ ]:
# # for low cardinality columns in our test dataset
# high_cardinality_X_test = X_test[list(high_cardinality_X_test_dict.keys())]
# high_cardinality_X_test.columns

In [ ]:
# # encoding for low cardinality
# low_cardinality_X_test = one_hot_encode.transform(low_cardinality_X_test)

In [ ]:
# # encoding for high cardinality
# high_cardinality_X_test = ordinal_encode.transform(high_cardinality_X_test)

In [ ]:
# # merging the encoded columns in our test data

# frames_test =[low_cardinality_X_test, high_cardinality_X_test]

# X_test = pd.concat(frames_test, axis = 1)

In [ ]:
# # imputing values on x test using the mean
# X_test = imputer.transform(X_test)

In [ ]:
# # scaling the x test
# X_test = scaler.transform(X_test)

Let's Visualize our data

In [ ]:
!pip install geopandas

In [ ]:
# let's make our imports
from pylab import rcParams
rcParams['figure.figsize'] = 30, 20
import geopandas

In [ ]:
# let's visualize the data

import geopandas

gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.longitude, df.latitude))

functional = gdf.where(gdf['status_group'] == 'functional')
repair = gdf.where(gdf['status_group'] == 'functional needs repair')
broken = gdf.where(gdf['status_group'] == 'non functional')

world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

# We restrict to Africa
ax = world[world.continent == 'Africa'].plot(
    color='gray', edgecolor='black')

ax.scatter(functional['longitude'], functional['latitude'],
           c='green',alpha=.5, s=3)

ax.scatter(repair['longitude'], repair['latitude'],
           c='blue', alpha=.5, s=5)

ax.scatter(broken['longitude'], broken['latitude'],
           c='red', alpha=.5, s=5)
plt.title("Map of Pump Distributions, Green-Functional, Blue-Repair, Red-Broken", fontsize = 25)

plt.ylim(-12, 0)
plt.xlim(28,41)

plt.show()

Modelling

In [ ]:
X_matrix, y_vector = features, df['status_group']

In [ ]:
majority_class = y_vector.mode()
y_vector.value_counts(normalize=True)

In [ ]:
from sklearn.metrics import accuracy_score
majority_prediction = [majority_class] * len(y_vector)
accuracy_score(y_vector, majority_prediction)

In [ ]:
df.columns

Decision Tree Classifier

In [ ]:
# let's import decision trees classifier

from sklearn.datasets import make_classification
from sklearn import tree
from sklearn.model_selection import train_test_split
 
X, y = make_classification(100, 5, n_classes = 2, shuffle = True, random_state= 10)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = True, random_state=1)

model = tree.DecisionTreeClassifier()
model = model.fit(X_train, y_train)

predicted_value = model.predict(X_test)
print(predicted_value)
#%%
tree.plot_tree(model)

zeroes = 0
ones = 0
for i in range(0,len(y_train)):
    if y_train[i] == 0:
        zeroes +=1
    else:
        ones +=1
      
print(zeroes)
print(ones)

val = 1 - ((zeroes/70)*2 + (ones/70)*2)
print("Gini : -",val)
 
match = 0
UnMatch = 0
 
for i in range(30):
    if predicted_value[i] == y_test[i]:
        match += 1
    else:
        UnMatch += 1
         
accuracy = match/30
print("Accuracy is: ",accuracy)

Gini Index is a score that evaluates how accurate a split is among the classified groups. Gini index evaluates a score in the range between 0 and 1, where 0 is when all observations belong to one class, and 1 is a random distribution of the elements within classes. The gini score attained implies that the model is accurate with random distribution of values.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(max_depth=20)
decision_tree.fit(X_matrix, y_vector)

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

def con_matrix_analysis(model):
  x = model.predict(features)
  y = y_vector
  
  print(classification_report(y, x,
        target_names=['Functional', 'Needs Repair', 'Not-Functional']))

  con_matrix = pd.DataFrame(confusion_matrix(y, x), 
             columns=['Predicted Functional', 'Predicted Needs Repair', 'Predicted Not-Functional'], 
             index=['Actual Functional', 'Actual Needs Repair', 'Actual Not-Functional'])
                            
  sns.heatmap(data=con_matrix, cmap='cool')
  plt.show();
  return con_matrix

In [ ]:
rcParams['figure.figsize'] = 15, 10

In [ ]:
con_matrix_analysis(decision_tree)

K-Nearest neighbor

In [ ]:
X_train = features
y_train = 
X_test = X_test
y_test = 

In [ ]:
features.head()